## Import libraries

In [245]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Import 'Shopee' dataset

In [248]:
df = pd.read_csv('Shopee Product Daily Mini Cube.csv')

In [250]:
df.columns

Index(['product_id', 'product_status', 'statistic_date', 'product_price',
       'product_price_before_discount', 'product_discount', 'product_stock',
       'is_on_flash_sale', 'stock_changed', 'new_items_sold', 'new_like_count',
       'new_view_count', 'new_comment_count', 'product_avg_rating',
       'new_rating_count', 'current_product_price', 'product_name',
       'product_image', 'currency', 'product_current_status', 'product_type',
       'product_category_id', 'product_current_price',
       'product_current_price_before_discount', 'shopee_verified_product',
       'product_current_stock', 'is_currently_on_flash_sale',
       'tier_variations', 'tier_variations_type', 'tier_variations_count',
       'product_current_avg_rating', 'total_rating_count',
       'rating_count_details', 'rating_count_with_context',
       'rating_count_with_image', 'product_url', 'first_live_date',
       'last_live_date', 'max_product_price', 'min_product_price',
       'max_product_discount', 'mi

In [252]:
df.head()

,product_id,product_status,statistic_date,product_price,product_price_before_discount,product_discount,product_stock,is_on_flash_sale,stock_changed,new_items_sold,...,shop_cancellation_rate,shop_total_products,follower_count,response_rate,response_time,preparation_time,following_count,shopurl,shop_last_inserted_date,last_updated_date
0,1037344035,normal,8/16/2021,329000,449000,0,8,0,0,0,...,7,131,90072,100,6554,357779,2,https://shopee.vn/tplink_officialstore,11/5/2021,10/26/2021
1,1037344035,normal,8/17/2021,369000,449000,0,6,0,-2,0,...,7,131,90072,100,6554,357779,2,https://shopee.vn/tplink_officialstore,11/5/2021,10/26/2021
2,1037344035,normal,8/18/2021,369000,449000,0,6,0,0,2,...,7,131,90072,100,6554,357779,2,https://shopee.vn/tplink_officialstore,11/5/2021,10/26/2021
3,1037344035,normal,8/19/2021,369000,449000,0,6,0,0,0,...,7,131,90072,100,6554,357779,2,https://shopee.vn/tplink_officialstore,11/5/2021,10/26/2021
4,1037344035,normal,8/20/2021,369000,449000,0,6,0,0,1,...,7,131,90072,100,6554,357779,2,https://shopee.vn/tplink_officialstore,11/5/2021,10/26/2021


## Clean product name start with discount code

In [255]:
import re
pd.set_option('display.max_rows', None)

def clean_product_name(name):
    if pd.isna(name):
        return ''
    
    name = name.strip()
    
    if name.startswith('['):
        parts = name.split(']', 1)
        return parts[1].strip() if len(parts) > 1 else ''
    else:
        return name

df['productname_cleaned'] = df['product_name'].apply(clean_product_name)

In [257]:
df['productname_cleaned'] = df['productname_cleaned'].str.rsplit('-', n=1).str[0]

In [259]:
import pandas as pd
import unicodedata

# -------------------------------
# 1. Normalize function
# -------------------------------
def normalize_text(text):
    if not isinstance(text, str):
        return ''
    return unicodedata.normalize('NFC', text).lower().strip()

# -------------------------------
# 2. Full cleaned keyword list
key_words = [
    'màn hình', 'usb', 'loa', 'bộ mở rộng sóng wifi', 'airtag', 'bảng vẽ điện tử',
    'bộ phát wifi', 'robot vacuum', 'iphone', 'áo thun', 'bộ chuyển đổi',
    'apple watch', 'watch fit + freebuds', 'chuột', 'bộ ấm trà', 'áo mưa',
    'camera', 'phím cơ', 'vòng đeo', 'lót tay', 'đồng hồ', 'sạc',
    'cân điện tử', 'cáp màn hình', 'máy lọc không khí', 'router', 'webcam',
    'cáp chuyển đổi', 'hệ thống wifi', 'bo mạch chủ', 'bàn phím', 'laptop',
    'máy giặt', 'bộ chia tín hiệu', 'máy hút bụi', 'tủ lạnh', 'tai nghe',
    'tivi', 'bộ chia mạng', 'điện thoại', 'máy tính', 'dây đeo apple watch',
    'ipad', 'bình giữ nhiệt', 'bóng đèn', 'bút trình chiếu', 'balo',
    'tay cầm chơi game không dây', 'máy lạnh', 'bộ sản phẩm huawei', 'pc',
    'hbs', 'thiết bị truyền tải nhạc', 'bộ định tuyến', 'lò vi sóng',
    'dây cáp', 'macbook', 'robot', 'bơm cầm tay', 'mouse', 'găng tay',
    'bộ cấp nguồn', 'hệ thống mesh', 'pencil', 'cáp dữ liệu',
    'bàn di cao cấp', 'bộ mở rộng sóng', 'máy chiếu', 'cần lái game',
    'apple tv', 'máy nâng cơ', 'thiết bị định tuyến mạng không dây',
    'thiết bị đo nhiệt độ và độ ẩm', 'điều khiển', 'nón bảo hiểm',
    'apple mac mini', 'combo', 'phần mềm microsoft office 365',
    'earphones', 'bộ mở rộng wifi', 'hộp gel dưỡng ẩm',
    'giftset tp', 'smart oled tv',
       'ổ cắm thông minh', 'tay cầm chơi game)',
       'bộ mở rộng internet', 'apple airpods pro',
       'nescafé mocha latte',
       'airpods',
       'phần mềm microsoft office', 'tay cầm chơi game'
]

# Normalize the keywords
key_words = [normalize_text(kw) for kw in key_words]

df['productname_normalized'] = df['productname_cleaned'].apply(normalize_text)


def find_category(text):
    for kw in key_words:
        if kw in text:
            return kw
    return None


df['category'] = df['productname_normalized'].apply(find_category)

In [275]:
df['category'] = df['category'].fillna('laptop')

In [277]:
df.loc[df['productname_normalized'] == 'v', 'category'] = 'đồng hồ'

In [279]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36970 entries, 0 to 36969
Data columns (total 75 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   product_id                             36970 non-null  int64  
 1   product_status                         36970 non-null  object 
 2   statistic_date                         36970 non-null  object 
 3   product_price                          36970 non-null  int64  
 4   product_price_before_discount          36970 non-null  int64  
 5   product_discount                       36970 non-null  int64  
 6   product_stock                          36970 non-null  int64  
 7   is_on_flash_sale                       36970 non-null  int64  
 8   stock_changed                          36970 non-null  int64  
 9   new_items_sold                         36970 non-null  int64  
 10  new_like_count                         36970 non-null  int64  
 11  ne

In [281]:
df['category'].unique()

array(['bộ chia tín hiệu', 'chuột', 'tai nghe', 'tivi', 'usb', 'màn hình',
       'bộ chia mạng', 'loa', 'apple watch', 'bộ mở rộng sóng wifi',
       'máy tính', 'điện thoại', 'ipad', 'airtag', 'laptop',
       'bảng vẽ điện tử', 'máy giặt', 'bo mạch chủ', 'sạc', 'máy hút bụi',
       'bộ phát wifi', 'tủ lạnh', 'robot vacuum', 'bàn phím', 'iphone',
       'áo thun', 'hệ thống wifi', 'watch fit + freebuds',
       'bộ sản phẩm huawei', 'bình giữ nhiệt', 'bóng đèn',
       'cáp chuyển đổi', 'bút trình chiếu', 'webcam', 'router', 'balo',
       'đồng hồ', 'phím cơ', 'tay cầm chơi game không dây',
       'máy lọc không khí', 'máy lạnh', 'camera', 'pc', 'hbs',
       'bộ cấp nguồn', 'thiết bị truyền tải nhạc', 'bộ định tuyến',
       'lò vi sóng', 'cân điện tử', 'bộ chuyển đổi', 'dây cáp', 'macbook',
       'robot', 'cần lái game', 'giftset tp', 'bộ mở rộng sóng',
       'apple tv', 'vòng đeo', 'hệ thống mesh', 'smart oled tv',
       'máy nâng cơ', 'ổ cắm thông minh', 'tay cầm chơi game',

In [283]:
vietnamese_to_english = {
    'bộ chia tín hiệu': 'Signal Splitter',
    'chuột': 'Mouse',
    'tai nghe': 'Headphones',
    'tivi': 'TV',
    'usb': 'USB',
    'màn hình': 'Monitor',
    'bộ chia mạng': 'Network Splitter',
    'loa': 'Speaker',
    'apple watch': 'Watch',
    'bộ mở rộng sóng wifi': 'WiFi Extender',
    'máy tính': 'Computer',
    'điện thoại': 'Phone',
    'ipad': 'iPad',
    'airtag': 'AirTag',
    'laptop': 'Laptop',
    'bảng vẽ điện tử': 'Drawing Tablet',
    'máy giặt': 'Washing Machine',
    'bo mạch chủ': 'Motherboard',
    'sạc': 'Charger',
    'máy hút bụi': 'Vacuum Cleaner',
    'bộ phát wifi': 'WiFi Router',
    'tủ lạnh': 'Refrigerator',
    'robot vacuum': 'Robot Vacuum',
    'bàn phím': 'Keyboard',
    'iphone': 'Phone',
    'áo thun': 'T-Shirt',
    'hệ thống wifi': 'WiFi System',
    'watch fit + freebuds': 'Watch Fit + FreeBuds',
    'bộ sản phẩm huawei': 'Product Set',
    'bình giữ nhiệt': 'Thermos',
    'bóng đèn': 'Light Bulb',
    'cáp chuyển đổi': 'Conversion Cable',
    'bút trình chiếu': 'Presenter Pen',
    'webcam': 'Webcam',
    'router': 'Router',
    'balo': 'Backpack',
    'đồng hồ': 'Watch',
    'phím cơ': 'Keyboard',
    'tay cầm chơi game không dây': 'Wireless Game Controller',
    'máy lọc không khí': 'Air Purifier',
    'máy lạnh': 'Air Conditioner',
    'camera': 'Camera',
    'pc': 'PC',
    'hbs': 'HBS',
    'bộ cấp nguồn': 'Power Supply',
    'thiết bị truyền tải nhạc': 'Music Transmission Device',
    'bộ định tuyến': 'Router',
    'lò vi sóng': 'Microwave',
    'cân điện tử': 'Electronic Scale',
    'bộ chuyển đổi': 'Converter',
    'dây cáp': 'Cable',
    'macbook': 'MacBook',
    'robot': 'Robot Vacuum',
    'cần lái game': 'Game Steering Wheel',
    'giftset tp': 'Gift Set',
    'bộ mở rộng sóng': 'Signal Extender',
    'apple tv': 'TV',
    'vòng đeo': 'Bracelet',
    'hệ thống mesh': 'Mesh System',
    'smart oled tv': 'Smart OLED TV',
    'máy nâng cơ': 'Lifting Machine',
    'ổ cắm thông minh': 'Smart Plug',
    'tay cầm chơi game': 'Game Controller',
    'bộ mở rộng internet': 'Internet Extender',
    'bàn di cao cấp': 'Mouse Pad',
    'apple airpods pro': 'AirPods',
    'combo': 'Combo',
    'apple mac mini': 'Macbook',
    'phần mềm microsoft office 365': 'Microsoft Office Software',
    'thiết bị định tuyến mạng không dây': 'Wireless Network Router',
    'cáp dữ liệu': 'Data Cable',
    'bộ mở rộng wifi': 'WiFi Extender',
    'mouse': 'Mouse',
    'nescafé mocha latte': 'Nescafé Mocha Latte',
    'airpods': 'AirPods',
    'earphones': 'Earphones',
    'găng tay': 'Gloves',
    'máy chiếu': 'Projector',
    'pencil': 'Pencil',
    'điều khiển': 'Remote Control',
    'thiết bị đo nhiệt độ và độ ẩm': 'Temperature and Humidity Sensor',
    'phần mềm microsoft office': 'Microsoft Office Software',
    'bơm cầm tay': 'Hand Pump',
    'lót tay': 'Palm Pad',
    'nón bảo hiểm': 'Helmet',
    'áo mưa': 'Raincoat',
    'bộ ấm trà': 'Tea Set'
}

# Map the existing 'category' column to English names in new column 'category_en'
df['category_en'] = df['category'].map(vietnamese_to_english)

df['category_en'] = df['category_en'].fillna('Unknown')


In [287]:
df['category_en'].unique()

array(['Signal Splitter', 'Mouse', 'Headphones', 'TV', 'USB', 'Monitor',
       'Network Splitter', 'Speaker', 'Watch', 'WiFi Extender',
       'Computer', 'Phone', 'iPad', 'AirTag', 'Laptop', 'Drawing Tablet',
       'Washing Machine', 'Motherboard', 'Charger', 'Vacuum Cleaner',
       'WiFi Router', 'Refrigerator', 'Robot Vacuum', 'Keyboard',
       'T-Shirt', 'WiFi System', 'Watch Fit + FreeBuds', 'Product Set',
       'Thermos', 'Light Bulb', 'Conversion Cable', 'Presenter Pen',
       'Webcam', 'Router', 'Backpack', 'Wireless Game Controller',
       'Air Purifier', 'Air Conditioner', 'Camera', 'PC', 'HBS',
       'Power Supply', 'Music Transmission Device', 'Microwave',
       'Electronic Scale', 'Converter', 'Cable', 'MacBook',
       'Game Steering Wheel', 'Gift Set', 'Signal Extender', 'Bracelet',
       'Mesh System', 'Smart OLED TV', 'Lifting Machine', 'Smart Plug',
       'Game Controller', 'Internet Extender', 'Mouse Pad', 'AirPods',
       'Combo', 'Macbook', 'Microsoft O

In [291]:
df = df.drop(columns = ['productname_normalized', 'category', 'productname_normalized'])

In [293]:
df.head()

,product_id,product_status,statistic_date,product_price,product_price_before_discount,product_discount,product_stock,is_on_flash_sale,stock_changed,new_items_sold,...,follower_count,response_rate,response_time,preparation_time,following_count,shopurl,shop_last_inserted_date,last_updated_date,productname_cleaned,category_en
0,1037344035,normal,8/16/2021,329000,449000,0,8,0,0,0,...,90072,100,6554,357779,2,https://shopee.vn/tplink_officialstore,11/5/2021,10/26/2021,Bộ Chia Tín Hiệu Switch TP-Link TL-SG1005D 5 c...,Signal Splitter
1,1037344035,normal,8/17/2021,369000,449000,0,6,0,-2,0,...,90072,100,6554,357779,2,https://shopee.vn/tplink_officialstore,11/5/2021,10/26/2021,Bộ Chia Tín Hiệu Switch TP-Link TL-SG1005D 5 c...,Signal Splitter
2,1037344035,normal,8/18/2021,369000,449000,0,6,0,0,2,...,90072,100,6554,357779,2,https://shopee.vn/tplink_officialstore,11/5/2021,10/26/2021,Bộ Chia Tín Hiệu Switch TP-Link TL-SG1005D 5 c...,Signal Splitter
3,1037344035,normal,8/19/2021,369000,449000,0,6,0,0,0,...,90072,100,6554,357779,2,https://shopee.vn/tplink_officialstore,11/5/2021,10/26/2021,Bộ Chia Tín Hiệu Switch TP-Link TL-SG1005D 5 c...,Signal Splitter
4,1037344035,normal,8/20/2021,369000,449000,0,6,0,0,1,...,90072,100,6554,357779,2,https://shopee.vn/tplink_officialstore,11/5/2021,10/26/2021,Bộ Chia Tín Hiệu Switch TP-Link TL-SG1005D 5 c...,Signal Splitter


In [303]:
df.shape

(36970, 73)

In [305]:
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.replace(r'[\r\n]+', ' ', regex=True)  # remove line breaks
    df[col] = df[col].str.replace(r'[^\x00-\x7F]+', '', regex=True)  # remove non-ASCII chars

In [307]:
import csv
df.to_csv('shopee_cat_en.csv', index=False, encoding='utf-8-sig', quoting=csv.QUOTE_ALL)
